In [ ]:
import numpy as np 
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
from pylabel import importer
from distutils.dir_util import copy_tree
import matplotlib.patches as patches
import matplotlib.patches as mpatches
from IPython.display import Image, display
from ultralytics import YOLO
from pathlib import Path
import albumentations as A
from albumentations.pytorch import ToTensorV2

In [ ]:
# Step 3: Data Augmentation
def augment_image(image, bbox, class_id):
    augmentations = A.Compose([
        A.HorizontalFlip(p=0.5),
        A.RandomBrightnessContrast(p=0.5),
        A.ShiftScaleRotate(shift_limit=0.0625, scale_limit=0.1, rotate_limit=15, p=0.5),
        A.GaussianBlur(p=0.3),
        ToTensorV2()
    ], bbox_params=A.BboxParams(format='yolo', label_fields=['class_id']))
    augmented = augmentations(image=image, bboxes=bbox, class_id=class_id)
    return augmented['image'], augmented['bboxes'], augmented['class_id']


In [ ]:
# Step 4: Paths
project_path = Path("/home/d356/ARVR")  # Base project path
data_yaml = project_path /"DATA/data.yaml"  # Path to data.yaml
weights_path = project_path /"MODEL/yolo11l.pt"  # Path to pre-trained weights
output_dir = project_path /"runs/train/YOLOv11_Improved"

# Step 5: Define and Load Model
model = YOLO(str(weights_path))  # pre-trained weights

In [ ]:
# Step 6: Train Model with Improved Parameters
model.train(
    data=str(data_yaml),        # Path to data.yaml
    epochs=300,                 # Train for more epochs
    batch=8,                    # Adjust batch size based on your hardware
    imgsz=640,                  # Larger image size for better detection
    optimizer='Adam',           # Try Adam optimizer for better convergence
    project=str(output_dir),    # Save results to a specific directory
    name="YOLOv11_Improved",    # Name of the experiment
    lr0=0.01,                   # Initial learning rate
    weight_decay=0.0005,        # Weight decay for regularization
    device=0,                   # GPU index (use "cpu" for CPU)
    patience=10,                # Early stopping patience
    cos_lr=True                 # Use cosine learning rate scheduler
)

# Step 7: Evaluate Model
metrics = model.val()
print("Validation Results:", metrics)

Ultralytics 8.3.63 🚀 Python-3.11.9 torch-2.4.0+cu124 CUDA:0 (NVIDIA GeForce RTX 2080 SUPER, 7965MiB)
engine/trainer: task=detect, mode=train, model=/home/d356/ARVR/MODEL/yolo11l.pt, data=/home/d356/ARVR/DATA/data.yaml, epochs=300, time=None, patience=100, batch=8, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=/home/d356/ARVR/runs/train/YOLOv11_Improved, name=YOLOv11_Improved3, exist_ok=False, pretrained=True, optimizer=Adam, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False

E0000 00:00:1737297655.807936  759848 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1737297655.812297  759848 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


Overriding model.yaml nc=80 with nc=7

                   from  n    params  module                                       arguments                     
  0                  -1  1      1856  ultralytics.nn.modules.conv.Conv             [3, 64, 3, 2]                 
  1                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  2                  -1  2    173824  ultralytics.nn.modules.block.C3k2            [128, 256, 2, True, 0.25]     
  3                  -1  1    590336  ultralytics.nn.modules.conv.Conv             [256, 256, 3, 2]              
  4                  -1  2    691712  ultralytics.nn.modules.block.C3k2            [256, 512, 2, True, 0.25]     
  5                  -1  1   2360320  ultralytics.nn.modules.conv.Conv             [512, 512, 3, 2]              
  6                  -1  2   2234368  ultralytics.nn.modules.block.C3k2            [512, 512, 2, True]           
  7                  -1  1   2360320  ultralytics

train: Scanning /home/d356/ARVR/DATA/train/labels.cache... 642 images, 0 backgrounds, 0 corrupt: 100%|██████████| 642/64

albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, num_output_channels=3, method='weighted_average'), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))


...


300 epochs completed in 1.621 hours.
Optimizer stripped from /home/d356/ARVR/runs/train/YOLOv11_Improved/YOLOv11_Improved3/weights/last.pt, 51.2MB
Optimizer stripped from /home/d356/ARVR/runs/train/YOLOv11_Improved/YOLOv11_Improved3/weights/best.pt, 51.2MB

Validating /home/d356/ARVR/runs/train/YOLOv11_Improved/YOLOv11_Improved3/weights/best.pt...
Ultralytics 8.3.63 🚀 Python-3.11.9 torch-2.4.0+cu124 CUDA:0 (NVIDIA GeForce RTX 2080 SUPER, 7965MiB)
YOLO11l summary (fused): 464 layers, 25,284,709 parameters, 0 gradients, 86.6 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 5/5 [00:00<00


                   all         68        117      0.939      0.914      0.962      0.676
                    F1         25         25          1      0.799       0.92      0.624
                 Green          3          3      0.937          1      0.995       0.55
                    K1         17         17          1      0.701      0.872      0.715
                    Q1         32         48      0.938      0.947      0.986      0.889
                   Red          5          5      0.823          1      0.995      0.504
                    T1          9          9      0.876          1      0.973      0.801
                    X1         10         10          1       0.95      0.995      0.649
Speed: 0.2ms preprocess, 8.2ms inference, 0.0ms loss, 1.4ms postprocess per image
Results saved to /home/d356/ARVR/runs/train/YOLOv11_Improved/YOLOv11_Improved3
Ultralytics 8.3.63 🚀 Python-3.11.9 torch-2.4.0+cu124 CUDA:0 (NVIDIA GeForce RTX 2080 SUPER, 7965MiB)
YOLO11l summary (fused): 4

val: Scanning /home/d356/ARVR/DATA/valid/labels.cache... 68 images, 0 backgrounds, 0 corrupt: 100%|██████████| 68/68 [00
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 9/9 [00:01<00


                   all         68        117      0.939      0.914      0.962      0.676
                    F1         25         25          1        0.8       0.92      0.626
                 Green          3          3      0.937          1      0.995       0.55
                    K1         17         17          1        0.7      0.872      0.716
                    Q1         32         48      0.938       0.95      0.986      0.889
                   Red          5          5      0.823          1      0.995      0.504
                    T1          9          9      0.875          1      0.973      0.801
                    X1         10         10          1       0.95      0.995      0.649
Speed: 0.2ms preprocess, 16.2ms inference, 0.0ms loss, 1.2ms postprocess per image
Results saved to /home/d356/ARVR/runs/train/YOLOv11_Improved/YOLOv11_Improved32
Validation Results: ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1, 2, 3, 4, 5, 6]

In [ ]:
metrics = model.val()

Ultralytics 8.3.63 🚀 Python-3.11.9 torch-2.4.0+cu124 CUDA:0 (NVIDIA GeForce RTX 2080 SUPER, 7965MiB)


val: Scanning /home/d356/ARVR/DATA/valid/labels.cache... 68 images, 0 backgrounds, 0 corrupt: 100%|██████████| 68/68 [00
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 9/9 [00:01<00


                   all         68        117      0.939      0.914      0.962      0.676
                    F1         25         25          1        0.8       0.92      0.626
                 Green          3          3      0.937          1      0.995       0.55
                    K1         17         17          1        0.7      0.872      0.716
                    Q1         32         48      0.938       0.95      0.986      0.889
                   Red          5          5      0.823          1      0.995      0.504
                    T1          9          9      0.875          1      0.973      0.801
                    X1         10         10          1       0.95      0.995      0.649
Speed: 0.2ms preprocess, 15.6ms inference, 0.0ms loss, 1.1ms postprocess per image
Results saved to /home/d356/ARVR/runs/train/YOLOv11_Improved/YOLOv11_Improved35
